In [41]:
import numpy as np
import gym
from numpy.random import choice
import random
from scipy.interpolate import interp1d

env = gym.make("Taxi-v2")

[2017-03-02 10:24:27,134] Making new env: Taxi-v2


In [18]:
def next_action(actions, e):
    n = len(actions)
    
    if random.random() < e:
        return random.randint(0, n-1)
    else:
        return np.argmax(actions)

In [27]:
n_actions = env.action_space.n
n_states = env.observation_space.n
Q = np.zeros([n_states, n_actions])

y = 0.99
k = 10000.0

In [39]:
print(Q.shape)
print(Q)

(500, 6)
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.56902818e+02   6.13287522e+02   5.09741634e+02   5.57033488e+02
    1.85011814e+03   7.14191937e+02]
 [  2.89092077e+02   3.85448415e+02   4.44235363e+02   6.51289420e+02
    1.88820141e+03   2.72870382e+02]
 ..., 
 [  6.07760860e+00  -1.88407423e+00  -2.03464663e+00   7.68239860e+02
   -7.69614949e+00  -9.05846201e+00]
 [ -4.49698682e+00  -4.54612075e+00  -4.54086614e+00   3.70445884e+02
   -1.09891596e+01  -1.20160893e+01]
 [ -1.68700710e+00   1.63691408e+01  -1.76151358e+00   1.40799605e+03
   -1.06643479e+01  -1.07296442e+01]]


In [42]:
print(env.observation_space.n)

500


In [29]:
r_total = 0
e_fun = interp1d([0, 40000], [1, 0.01], bounds_error=False, fill_value=0.01)
learning_rate_f = lambda t: k / (k + t)
global_step = 0

for t in range(200000):
    s = env.reset()
    
    while True:
        global_step += 1
        learning_rate = learning_rate_f(global_step)
        e = e_fun(global_step)
        
        a = next_action(Q[s], e)

        s1, r, done, info = env.step(a)
        r_total += r

        Q[s, a] += learning_rate * (r + y * np.max(Q[s1]) - Q[s, a])
        
        s = s1
        
        if done:
            break
            
    if t % 5000 == 0:
        print(r_total / 5000.0)
        r_total = 0

-2.1754
-8.2996
7.8674
7.909
7.9202
7.9064
7.908
7.891
7.9224
7.8918
7.978


KeyboardInterrupt: 

In [43]:
Q

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.56902818e+02,   6.13287522e+02,   5.09741634e+02,
          5.57033488e+02,   1.85011814e+03,   7.14191937e+02],
       [  2.89092077e+02,   3.85448415e+02,   4.44235363e+02,
          6.51289420e+02,   1.88820141e+03,   2.72870382e+02],
       ..., 
       [  6.07760860e+00,  -1.88407423e+00,  -2.03464663e+00,
          7.68239860e+02,  -7.69614949e+00,  -9.05846201e+00],
       [ -4.49698682e+00,  -4.54612075e+00,  -4.54086614e+00,
          3.70445884e+02,  -1.09891596e+01,  -1.20160893e+01],
       [ -1.68700710e+00,   1.63691408e+01,  -1.76151358e+00,
          1.40799605e+03,  -1.06643479e+01,  -1.07296442e+01]])

In [45]:
s = env.reset()
done = False
for i in range(1000):
    a = next_action(Q[s], 0)
    s, r, done, info = env.step(a)
    env.render()
    print(r)
    print(s)
    print("")
    
    if done:
        break

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
-1
182

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
-1
162

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
-1
142

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
-1
122

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
-1
102

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
-1
2

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
-1
18

+---------+
|R: | : :G|
|_: : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
-1
118

+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
-1
218

+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (S